
# Discover Cost-Efficient AI Customer Service Agents with NVIDIA Data Flywheel Blueprint
[![ Click here to deploy.](https://brev-assets.s3.us-west-1.amazonaws.com/nv-lb-dark.svg)](https://brev.nvidia.com/launchable/deploy?launchableID=env-2wggjBvDlVp4pLQD8ytZySh5m8W)

A data flywheel is a feedback loop where data collected from interactions or processes is used to continuously refine AI models, which in turn generates better outcomes and more valuable data. In this notebook, you will learn how to use the Data Flywheel Foundational Blueprint and the [Agent Intelligence (AIQ) toolkit](https://docs.nvidia.com/aiqtoolkit/latest/index.html) to continuously discover and promote more cost-efficient agents for an [AI virtual customer service assistant](https://build.nvidia.com/nvidia/ai-virtual-assistant-for-customer-service). The [Agent Intelligence (AIQ) toolkit](https://docs.nvidia.com/aiqtoolkit/latest/index.html) is a flexible, lightweight, and unifying library that allows you to easily connect existing enterprise agents to data sources and tools across any framework. For more information, please reference the [documentation](https://docs.nvidia.com/aiqtoolkit/latest/index.html) and [GitHub project](https://github.com/NVIDIA/AIQToolkit/tree/develop).

### Data Flywheel Blueprint

![Data Flywheel Blueprint](../docs/images/data-flywheel-blueprint.png)


### AI Virtual Assistant for Customer Service

The primary customer service agent in the AI Virtual Assistant uses tool calling to route user queries to specialized assistants, including: 

- Product Q&A
- Order status verification
- Returns processing
- Small talk and casual engagement

The [Agent Intelligence toolkit](https://docs.nvidia.com/aiqtoolkit/latest/index.html) will automatically collect and transmit runtime logs and tool-calling data from these interactions to Data Flywheel, enabling you to leverage them as evaluation benchmarks and training data.
In this tutorial, you'll use this information to drive the flywheel process, fine-tuning smaller LLMs (such as `meta/llama-3.2-1B-instruct`, `meta/llama-3.2-3B-instruct`, `meta/llama-3.1-8B-instruct`) to match accuracy of the currently deployed model (`meta/llama-3.3-70B-instruct`).



## Interfacing with the Blueprint

The following diagram illustrates how admin tools and applications interact with the Flywheel Blueprint, which orchestrates logging, processing, and model management to enable continuous optimization.

![Arch](arch.png)

### Contents 

0. [Data Flywheel Setup](#0)
1. [Interact with baseline AI Virtual Assistant](#1)
2. [Runtime Data Flywheel Ingestion](#2)
3. [Create a Flywheel Job](#3)
4. [Monitor Job Status](#4)
5. [Redeploy AI Virtual Assistant](#5)
6. [Optional: Show Continuous Improvement](#6)

<a id="0"></a>
## Data Flywheel Setup

The Data Flywheel service is built on top of the [NeMo Microservices](https://docs.nvidia.com/nemo/microservices/latest/about/index.html). Before setting up the DataFlywheel service, ensure that NeMo Microservices is already deployed in your environment — it serves as a prerequisite for this workflow.

The DataFlywheel service itself is packaged as a set of Docker containers and can be brought up using Docker Compose.

In general, you can set up the Data Flywheel service by following the instructions provided in the [Quick Start Guide](https://github.com/NVIDIA-AI-Blueprints/data-flywheel/blob/main/docs/02-quickstart.md). 


If you want to quickly spin up the DataFlywheel service with minimal configuration, we recommend starting with the [Data Flywheel Blueprint Brev Launchable](https://brev.nvidia.com/launchable/deploy/now?launchableID=env-2wggjBvDlVp4pLQD8ytZySh5m8W) (see instructions below).


### NVIDIA Brev Launchable Setup Instructions

> **Important:** The instructions below apply **only** to users running this notebook via the Brev Launchable.

NVIDIA Brev is a developer-friendly platform that makes it easy to run, train, and deploy ML models on cloud GPUs without the hassle of setup—it comes preloaded with Python, CUDA, and Docker so you can get started fast. 

Brev Launchables are shareable, pre-preconfigured GPU environments that bundle your code, containers, and compute into one easy-to-launch link.

Please follow the steps below if you are using this notebook as part of the Brev Launchable.

**Step 1**: Set API Keys - [Generating NGC API Keys](https://docs.nvidia.com/ngc/gpu-cloud/ngc-private-registry-user-guide/index.html#generating-api-key).

In [ ]:
import os
os.environ['NGC_API_KEY'] = '<your_ngc_api_key>'
os.environ['NVIDIA_API_KEY'] = '<your_nim_api_key>'

**Step 2**: Clone the data flywheel repo and fetch data files.

In [ ]:
%%bash
git clone https://github.com/NVIDIA-AI-Blueprints/data-flywheel.git
cd data-flywheel
sudo apt-get update && sudo apt-get install -y git-lfs
git lfs install
git-lfs pull

**Step 3**: Set up paths and installs python dependencies for notebook.

In [ ]:
import sys
from pathlib import Path

notebook_dir = Path.cwd()
project_root = notebook_dir / "data-flywheel"
data_dir = project_root / "data"
sys.path.insert(0, str(project_root))
os.chdir(project_root)
print(f"Working directory changed to: {Path.cwd()}")

user_site = Path.home() / ".local" / "lib" / f"python{sys.version_info.major}.{sys.version_info.minor}" / "site-packages"
if str(user_site) not in sys.path:
    sys.path.append(str(user_site))
    print(f"Added user site-packages to sys.path: {user_site}")

%pip install --user elasticsearch==8.17.2 pydantic-settings>=2.9.1 pandas>=2.2.3 matplotlib==3.10.3

Import required libraries and configure pandas display options for better readability in notebook outputs

In [ ]:
import requests
import pandas as pd

pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)        # Width of the display in characters
pd.set_option('display.max_colwidth', None)  # Show full content of each cell

**Step 4**: Update `config/config.yaml` to use remote LLM as judge. By default, data flywheel blueprint deploys `LLama-3.3-70B-instruct` locally for LLM as a judge, which requires 4 GPUs. But for the launchable, we will choose the remote LLM judge and use the `LLama-3.3-70B-instruct` NIM hosted on [build.nvidia.com](https://build.nvidia.com/meta/llama-3_3-70b-instruct).

By default, only `Llama-3.2-1b-instruct` will be used in the flywheel but you can uncomment other models in the yaml file to include in the flywheel run. You can also change other config settings such as data split and training hyperparameters as desired



In [ ]:
import re
from textwrap import dedent

config_path = project_root / "config" / "config.yaml"

new_llm_block = dedent("""\
llm_judge_config:
  type: "remote"
  url: "https://integrate.api.nvidia.com/v1/chat/completions"
  model_id: "meta/llama-3.3-70b-instruct"
  api_key_env: "NGC_API_KEY"

""")

new_nims_block = dedent("""\
nims:
  - model_name: "meta/llama-3.2-1b-instruct"
    context_length: 8192
    gpus: 1
    pvc_size: 25Gi
    tag: "1.8.3"
    customization_enabled: true

  - model_name: "meta/llama-3.2-3b-instruct"
    context_length: 32768
    gpus: 1
    pvc_size: 25Gi
    tag: "1.8.3"
    customization_enabled: true

  - model_name: "meta/llama-3.1-8b-instruct"
    context_length: 32768
    gpus: 1
    pvc_size: 25Gi
    tag: "1.8.3"
    customization_enabled: true

""")

text = config_path.read_text()

def replace_block(yaml_text: str, key: str, new_block: str) -> str:
    pattern = rf"(?ms)^({re.escape(key)}:[\s\S]*?)(?=^\S|\Z)"
    return re.sub(pattern, new_block, yaml_text)

text = replace_block(text, "llm_judge_config", new_llm_block)
text = replace_block(text, "nims",              new_nims_block)

config_path.write_text(text)
print("config.yaml updated")

**Step 5**: Start data flywheel service, which involves first deploying the Nemo Microservices and then bring up the data flywheel service via docker compose. This step take some time.

In [ ]:
%%bash
set -e

log() {
  echo -e "\033[1;32m[INFO]\033[0m $1"
}

echo "$NGC_API_KEY" | docker login nvcr.io -u '$oauthtoken' --password-stdin
chmod +x scripts/deploy-nmp.sh scripts/run.sh

log "Starting Nemo Microservices deployment..."
./scripts/deploy-nmp.sh >> flywheel_deploy.log 2>&1
log "NMP deployed successfully!"

log "Starting data flywheel service..."
./scripts/run.sh >> flywheel_deploy.log 2>&1
log "Data flywheel service started successfully!"

**Step 6**: Install the AI Virtual Assistant Blueprint. This will take a few minutes to pull down and build containers the first time.

In [ ]:
%%bash
# Clone repository
git clone --branch aiq-dfw-integration https://github.com/NVIDIA-AI-Blueprints/ai-virtual-assistant.git
cd ai-virtual-assistant
# Install blueprint
echo "Building api-gateway-server..."
docker compose -f deploy/compose/docker-compose.aiq.yaml build api-gateway-server --quiet
echo "Building agent-chain-server..."
docker compose -f deploy/compose/docker-compose.aiq.yaml build agent-chain-server --quiet

In [ ]:
%%bash
# Download datasets
cd ai-virtual-assistant
echo "Downloading datasets..."
echo "Ingesting datasets..."
docker compose -f src/ingest_service/docker-compose.yaml run ingest-client

---

<a id="1"></a>
## Step 1: Interact with baseline AI Virtual Assistant

As an AIQ toolkit based agent, the AI Virtual Assistant can be deployed by YAML configuration and is seamlessly integrated with Data Flywheel. The most salient sections of this YAML file are illustrated below.

```yaml
general:
  use_uvloop: true
  logging:
    console:
      _type: console
      level: WARN

  telemetry:
    tracing: 
      dfw_elasticsearch:  # <- Simple configuration to ship runtime logs to Data Flywheel
        _type: nemo_dfw_elasticsearch
        endpoint: ${DATA_FLYWHEEL_ENDPOINT}
        client_id: ${CLIENT_ID}
        index: flywheel
      phoenix:  # <- Traces will also ship to a running Phoenix server
        _type: phoenix
        endpoint: ${PHOENIX_ENDPOINT}
        project: ai-virtual-assistant        

... # Omitting some sections for brevity

llms:
  tool_call_llm:
    _type: nim
    model_name: ${APP_TOOLCALL_LLM_MODELNAME}  # <- We start with a meta/llama-3.3-70b-instruct model
    base_url: ${APP_TOOLCALL_LLM_SERVERURL}
    temperature: 0.2
    top_p: 0.7
    max_tokens: 1024    
    api_key: ${NVIDIA_API_KEY}
  chat_llm:
    _type: nim
    model_name: ${APP_CHAT_LLM_MODELNAME}
    base_url: ${APP_CHAT_LLM_SERVERURL}
    temperature: 0.2
    top_p: 0.7
    max_tokens: 1024    
    api_key: ${NVIDIA_API_KEY}    

workflow:
  _type: aiva_agent
  tool_call_llm_name: tool_call_llm
  chat_llm_name: chat_llm

```

Deployment environment variables are passed through to the AIQ toolkit configuration object to drive runtime configuration settings. There are two important sections of this file that enable the Data Flywheel to improve application performance:

1. **Data Flywheel Integration** - AIQ toolkit provides configurable Data Flywheel integrations, allowing developers to stream runtime logs effortlessly and without additional development costs.
2. **LLM selection** - Configure the optimal LLM for your Agentic application based on insights from Data Flywheel, allowing for tailored performance and efficiency gains. Defaulting to meta/llama-3.3-70b-instruct, but easily adaptable to lighter-weight models.



**Deploy the Baseline AI Virtual Assistant**

Now, let's deploy AI Virtual Assistant agent using a `meta/llama-3.3-70b-instruct` model for both tool calling and general chat completions tasks.

In [ ]:
%%bash
cd ai-virtual-assistant
# Set our deployment parameters
export APP_TOOLCALL_LLM_MODELNAME=meta/llama-3.3-70b-instruct
export APP_CHAT_LLM_MODELNAME=meta/llama-3.3-70b-instruct
export APP_LLM_MODELENGINE=nvidia-ai-endpoints
export DFW_CLIENT_ID=aiq-ai-virtual-assistant

# Bring down the current AI Virtual Assistant agent service
echo "Bringing down the agent-chain-server"
docker compose -f deploy/compose/docker-compose.aiq.yaml down agent-chain-server
# Redeploy the AI Virtual Assistant
echo "Redeploying the agent-chain-server"
docker compose -f deploy/compose/docker-compose.aiq.yaml up --quiet-pull -d

## Interfacing with the Blueprint
![AIVA-UI](img/aiva-ui.png)

Let's now interact with AI Virtual Assistant through its user interface hosted at [http://localhost:3001](http://localhost:3001). A Phoenix server will also be available at [http://localhost:6006](http://localhost) for an intuitive view of all runtime traces.

---

<a id="2"></a>
## Step 2: Runtime Data Flywheel Ingestion


Since we have been interacting with the AI Virtual Assistant application, runtime traces have been transmitted to Data Flywheel. These data points are considered **ground truth**.

Ground truth data points are used to **evaluate** and **customize** more efficient models that can perform similarly to the current model. This customization process is analogous to a student-teacher distillation setup, where synthetic data generated from the teacher model is used to fine-tune a student model.

Each data point has the following schema:

| Field        | Type               | Description                                                         |
|--------------|--------------------|---------------------------------------------------------------------|
| `timestamp`  | `int` (epoch secs) | Time the request was issued                                         |
| `workload_id`| `str`              | Stable identifier for the logical task / route / agent node         |
| `client_id`  | `str`              | Identifier of the application or deployment that generated traffic  |
| `request`    | `dict`             | Exact [`openai.ChatCompletion.create`](https://platform.openai.com/docs/api-reference/chat/create) payload received by the model |
| `response`   | `dict`             | Exact `ChatCompletion` response returned by the model               |

The `request` uses the OpenAI `ChatCompletions` request format and contains the following attributes:

- `model` includes the Model ID used to generate the response.
- `messages` includes a `system` message as well as a `user` query.
- `tools` includes a list of functions and parameters available to the LLM to choose from, as well as their parameters and descriptions.

Let's have a look at the most recent AI Virtual Assistant runtime trace that has been ingested into Data Flywheel.

In [ ]:
!curl -X POST "http://localhost:9200/flywheel/_search" \
  -H 'Content-Type: application/json' \
  -d '{"size": 1, "sort": [{"timestamp": {"order": "desc"}}]}' | jq

This is great! In production, we would likely be collecting data for an extended period. To speed up the process, let's simulate collectioning these traces with a bulk ingestion so we can kick off the Data Flywheel.

In [ ]:
from src.scripts.load_test_data import load_data_to_elasticsearch

load_data_to_elasticsearch(file_path="aiva_primary_assistant_dataset.jsonl", workload_id="aiva_agent", client_id="aiq-ai-virtual-assistant")

---

<a id="3"></a>
## Step 3: Create a Flywheel Job

Initiate a Flywheel job by sending a POST request to the `/jobs` API. This triggers the workflow asynchronously.

In production environments, you can automate this process to run at scheduled intervals, in response to specific events, or on demand.

For this tutorial, we will target the primary customer service agent by setting the `workload_id` to "aiva_agent" and we will set `client_id` to "aiq-ai-virtual-assistant" which has 5079 data points.

In [ ]:
# Flywheel Orchestrator URL
API_BASE_URL = "http://0.0.0.0:8000"

response = requests.post(
    f"{API_BASE_URL}/api/jobs",
    json={"workload_id": "aiva_agent", "client_id": "aiq-ai-virtual-assistant"}
)

response.raise_for_status()
job_id = response.json()["id"]

print(f"Created job with ID: {job_id}")

---

<a id="4"></a>
## Step 4: Monitor Job Status

Submit a GET request to `/jobs/{job_id}` to retrieve the current status.

In [ ]:

def get_job_status(job_id):
    """Get the current status of a job."""
    response = requests.get(f"{API_BASE_URL}/api/jobs/{job_id}")
    response.raise_for_status()
    return response.json()

In [ ]:
get_job_status(job_id)

In the job status output, you will see the following metrics for evaluating the accuracy of tool calling:

- `function_name_accuracy`: This metric evaluates whether the LLM correctly predicts the function name.
    - **Definition**: It checks for an **exact match** between the predicted function name and the ground truth function name.
    - **Scoring**:
        - `1` if the predicted function name exactly matches the ground truth.
        - `0` otherwise.

- `function_name_and_args_accuracy (exact-match)`: This stricter metric checks whether **both** the function name and all associated function arguments are correctly predicted.
    - **Definition**: The prediction is considered correct **only** if:
        - The function name is an **exact match**, and
        - Every function argument is also an **exact match** to the ground truth.
    - **Scoring**:
        - `1` if both the function name and all arguments exactly match.
        - `0` otherwise.

    This measures the LLM's ability to generate an entirely accurate function call, including both the correct operation and the exact input values.

- `function_name_and_args_accuracy (LLM-judge)`: This metric uses a LLM to act as a "judge" and assess the correctness of the function call based on semantic meaning, particularly useful when arguments are complex or naturally rephrased.
    - **Definition**:
        - The function name must be an **exact match**.
        - For function arguments:
            - If an argument is simple and expected to match exactly (e.g., a user ID or fixed keyword), it must be an **exact match**.
            - If an argument is more complex (e.g., a user query or free-text input), **semantic similarity** is evaluated using the LLM-as-judge.
    - **Scoring**:
        - `1` if all criteria (function name match, and each argument passing either the exact match or semantic check) are satisfied.
        - `0` otherwise.

    This metric captures functional correctness even when the LLM rewrites or paraphrases input arguments, as long as the **intent and outcome remain accurate**.

To simplify the process and enable continuous monitoring, we defined a utility function `monitor_job` in `job_monitor_helper.py`:

- Periodically retrieve the job status
- Format the output into a table

This makes it easier to compare and analyze the results.

In [ ]:
from notebooks.job_monitor_helper import monitor_job

# Start monitoring the job with polling interval of 5s
monitor_job(api_base_url=API_BASE_URL, job_id=job_id, poll_interval=5)

You’ve now successfully completed a Flywheel run and can review the evaluation results to decide whether to promote the model. However, with only 300 data points, the customized `Llama-3.2-1B-instruct` is likely still limited in performance.

That said, the Data Flywheel operates as a self-reinforcing cycle—models continue to improve as more user interaction logs are collected. Below, we demonstrate how the model performance improves incrementally with additional data. 

Note that the Eval metrics shown in the figures are the `function_name_accuracy`.


![300dp](https://raw.githubusercontent.com/NVIDIA-AI-Blueprints/data-flywheel/main/notebooks/img/300dp.png)


**Flywheel run results at 300 data points**

![500dp](https://raw.githubusercontent.com/NVIDIA-AI-Blueprints/data-flywheel/main/notebooks/img/500dp.png)

**Flywheel run results at 500 data points**

![1000dp](https://raw.githubusercontent.com/NVIDIA-AI-Blueprints/data-flywheel/main/notebooks/img/1000dp.png)

**Flywheel run results at 1,000 data points**

With the improvement results demonstrated, you can now move on to Step 4 to run the Flywheel with additional data yourself.

---

<a id="5"></a>
## Step 5: Redeploy AI Virtual Assistant 

**Deploy the Customized NIM with the AI Virtual Assistant**
The flywheel job has trained LoRA adapters for our LLMs that can be redeployed as NIMs in our local cluster. Follow the steps to deploy this model with the AI Virtual Assistant application.

**Step 1**: Identify the model based on evaluation criteria.

**Step 2**: Identify the selected model's customization `nmp_uri`. It will have the following format:

`http://nemo.test/v1/customization/jobs/<CUSTOMIZATION_ID>`

Grab the `CUSTOMIZATION_ID` from this ouput.

**Step 3**: Deploy the NIM and corresponding LoRA adapter(s).

In [ ]:
%%bash
curl --location 'http://nemo.test/v1/deployment/model-deployments'
     --header 'Content-Type: application/json'
     --data '{
     "name": "<YOUR_MODEL_NAME>",
     "namespace": "meta",
     "config": {
        "model": "<YOUR_MODEL_NAME>",
        "nim_deployment": {
           "image_name": "nvcr.io/nim/<YOUR_MODEL_NAME>",
           "image_tag": "1.8.3",
           "pvc_size": "25Gi",
           "gpu": 1,
           "additional_envs": {}
           }
        }
     }'

In [ ]:
# Optionally delete the model deployment
# curl -X DELETE 'http://nemo.test/v1/deployment/model-deployments/<MODEL_NAMESPACE>\<MODEL_NAME>'
# EXAMPLE: curl -X DELETE 'http://nemo.test/v1/deployment/model-deployments/meta\meta\/llama-3.2-1b-instruct'

**Step 4**: Update the configuration and redeploy the AI Virtual Assistant.

In [ ]:
%%bash
export APP_LLM_MODELNAME=dfwbp/customized-<YOUR_MODEL_NAME>@<CUSTOMIZATION_ID>
export APP_LLM_MODELENGINE=nvidia-ai-endpoints
export APP_LLM_SERVERURL=http://nim.test/v1/chat/completions
export DFW_CLIENT_ID=aiq-aiva-customized

# Bring down the current AI Virtual Assistant agent service
echo "Bringing down the agent-chain-server"
docker compose -f deploy/compose/docker-compose.aiq.yaml down agent-chain-server
# Redeploy the AI Virtual Assistant
echo "Redeploying the agent-chain-server"
docker compose -f deploy/compose/docker-compose.aiq.yaml up --quiet-pull -d

**Step 5**: Experience the updated application hosted at [http://localhost:3001](http://localhost:3001) 

This time we are using a much smaller model, while preserving the quality of outputs! In summary, the our AIQ tookit based application has streamed groud truth traces directly to flywheel. Flywheel has enabled us to discover more optimimal models for our application. Redeploying this application provides a more cost optimized deployment and improved end user experience. 

This time we are using a much smaller model, while preserving the quality of outputs!

## Key Takeaways:

The **NVIDIA Agent Intelligence (AIQ) toolkit**, combined with a **data flywheel**, enables organizations to develop cost-effective, personalized, and adaptive AI applications that drive business success. Key takeaways from our experience include:

1. The Agent Intelligence (AIQ) toolkit provides a flexible and lightweight foundation for deploying production-ready agentic AI,
2. A data flywheel is essential for continuous improvement and adaptability, allowing organizations to refine offerings and meet tangible goals, and
3. By leveraging the AIQ toolkit and data flywheel, we were able to deploy a more cost-optimized application with improved end-user experience, while preserving output quality, demonstrating the potential for significant business benefits.

## Step 6: Show Continuous Improvement (Optional)

To extend the flywheel run with additional data, consider running the AIVA application for an extended period of time and/or bulk ingest some additional simulated runtime data to evaluate the impact of increased data volume on performance.

In [ ]:
response = requests.post(
    f"{API_BASE_URL}/api/jobs",
    json={"workload_id":  <YOUR_WORKLOAD_ID>, "client_id": <YOUR_CLIENT_ID>}
)

response.raise_for_status()
job_id = response.json()["id"]

print(f"Created job with ID: {job_id}")

In [ ]:
monitor_job(api_base_url=API_BASE_URL, job_id=job_id, poll_interval=5)

You should see some improvements of the customized model compared to the last run.

Assuming we have now collected even more data points, let's kick off another flywheel run by setting `client_id` to "aiva-3" which includes **1,000** records.

In [ ]:
response = requests.post(
    f"{API_BASE_URL}/api/jobs",
    json={"workload_id": <YOUR_WORKLOAD_ID>, "client_id": <YOUR_CLIENT_ID>}
)

response.raise_for_status()
job_id = response.json()["id"]

print(f"Created job with ID: {job_id}")

In [ ]:
monitor_job(api_base_url=API_BASE_URL, job_id=job_id, poll_interval=5)

After the run with 1,000 data points, we should observe the customized model’s score approaching 1.0. This indicates that the `LLama-3.2-1B-instruct` model achieves accuracy comparable to the much larger `LLama-3.3-70B-instruct` base model deployed in AI Virtual Assistant, while significantly reducing latency and compute usage thanks to its smaller size.